In [1]:
!pip install transformers datasets

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding, pipeline
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:

def load_data():
    dataset = load_dataset("amazon_polarity")
    return dataset


def preprocess_data(dataset, tokenizer):
    def tokenize_function(examples):
        return tokenizer(examples["content"], truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    return tokenized_datasets


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


def train_model(tokenized_datasets, tokenizer):
    model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


    train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(3000))
    test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1500))


    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        save_strategy="epoch",
        logging_dir="./logs",
        logging_steps=25,
        load_best_model_at_end=True,
        report_to="none"
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    print("Evaluation Results:", eval_results)

    return model

def analyze_sentiments(texts, sentiment_pipeline):


    label_map = {0: "NEGATIVE", 1: "POSITIVE"}
    results = []

    for text in texts:
        result = sentiment_pipeline(text)[0]
        label = label_map[int(result['label'][-1])]  # Map label to positive/negative
        confidence = f"{result['score']:.3f}"
        results.append({"TEXT": text, "SENTIMENT": label, "CONFIDENCE": confidence})

    return results



In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead")


Using GPU: Tesla T4


In [4]:

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset = load_data()


    train_texts = dataset["train"]["content"]
    train_labels = dataset["train"]["label"]
    label_map = {0: "negative", 1: "positive"}
    for i, (text, label) in enumerate(zip(train_texts[:5], train_labels[:5])):
        print(f"Review {i + 1}: {text}\nSentiment: {label_map[label]}\n")

    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    tokenized_datasets = preprocess_data(dataset, tokenizer)

    model = train_model(tokenized_datasets, tokenizer)


    sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

Map:   0%|          | 0/400000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-4c86fcd24a34>:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.274700,0.269524,0.892667,0.925352,0.858824,0.890847
2,0.154600,0.320447,0.896000,0.881101,0.920261,0.900256
3,0.099200,0.376661,0.892667,0.876559,0.918954,0.897256


Evaluation Results: {'eval_loss': 0.269524484872818, 'eval_accuracy': 0.8926666666666667, 'eval_precision': 0.9253521126760563, 'eval_recall': 0.8588235294117647, 'eval_f1': 0.8908474576271187, 'eval_runtime': 8.5803, 'eval_samples_per_second': 174.818, 'eval_steps_per_second': 10.955, 'epoch': 3.0}


In [5]:
texts = ["This product exceeded my expectations! The quality is fantastic, and the delivery was super fast."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': 'This product exceeded my expectations! The quality is fantastic, and the delivery was super fast.',
  'SENTIMENT': 'POSITIVE',
  'CONFIDENCE': '0.968'}]

In [6]:
texts = ["Delivery was late, and the item was nothing like the description. Very disappointed."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': 'Delivery was late, and the item was nothing like the description. Very disappointed.',
  'SENTIMENT': 'NEGATIVE',
  'CONFIDENCE': '0.970'}]

In [7]:
texts = ["The design is nice, but the functionality is average. Could use some improvements."]
analyze_sentiments(texts, sentiment_pipeline)


[{'TEXT': 'The design is nice, but the functionality is average. Could use some improvements.',
  'SENTIMENT': 'POSITIVE',
  'CONFIDENCE': '0.737'}]

In [8]:
texts = ["The product stopped working after a week. Waste of money. Do not buy this."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': 'The product stopped working after a week. Waste of money. Do not buy this.',
  'SENTIMENT': 'NEGATIVE',
  'CONFIDENCE': '0.973'}]

In [9]:
texts = ["I absolutely love this gadget! It works exactly as described and has made my life so much easier."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': 'I absolutely love this gadget! It works exactly as described and has made my life so much easier.',
  'SENTIMENT': 'POSITIVE',
  'CONFIDENCE': '0.974'}]

In [10]:
texts = ["Packaging was decent, and delivery was on time. Haven't tried the product yet."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': "Packaging was decent, and delivery was on time. Haven't tried the product yet.",
  'SENTIMENT': 'POSITIVE',
  'CONFIDENCE': '0.737'}]

In [11]:
texts = ["Terrible experience! The item arrived damaged, and the seller refused to provide a replacement."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': 'Terrible experience! The item arrived damaged, and the seller refused to provide a replacement.',
  'SENTIMENT': 'NEGATIVE',
  'CONFIDENCE': '0.966'}]

In [12]:
texts = ["The product was okay. Not great, but not bad either. You get what you pay for."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': 'The product was okay. Not great, but not bad either. You get what you pay for.',
  'SENTIMENT': 'POSITIVE',
  'CONFIDENCE': '0.784'}]

In [13]:
texts = ["The quality is awful. It broke on the first use, and the material feels really cheap."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': 'The quality is awful. It broke on the first use, and the material feels really cheap.',
  'SENTIMENT': 'NEGATIVE',
  'CONFIDENCE': '0.968'}]

In [14]:
texts = ["Worst customer service ever. They ignored my emails and refused to issue a refund."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': 'Worst customer service ever. They ignored my emails and refused to issue a refund.',
  'SENTIMENT': 'NEGATIVE',
  'CONFIDENCE': '0.970'}]

In [16]:
texts = ["Customer service was excellent. They responded quickly and resolved my issue without any hassle."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': 'Customer service was excellent. They responded quickly and resolved my issue without any hassle.',
  'SENTIMENT': 'POSITIVE',
  'CONFIDENCE': '0.927'}]

In [17]:
texts = ["Great value for the price. Highly recommend to anyone looking for a budget-friendly option."]
analyze_sentiments(texts, sentiment_pipeline)

[{'TEXT': 'Great value for the price. Highly recommend to anyone looking for a budget-friendly option.',
  'SENTIMENT': 'POSITIVE',
  'CONFIDENCE': '0.970'}]